<h2>
CSC 44700 - Introduction to Machine Learning<br/>
May 22, 2022<br/>
Professor: Erik Grimmelmann<br/>
Mark Natavio <br/>
Final Project Codes
</h2>

<hr/>

# Logistic Regession

For section, I will be using a Logistic regrssion model for text classification

#### Importing

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline

#### Data

In [2]:
df = pd.read_csv('/Users/marknatavio/Desktop/Final project/Codes/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-15",names=['Target','ID','Date','Flag','User','Text'])
df.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


#### Insights

In [3]:
print("Target values:\n"+str(df['Target'].unique()))
print("\nTarget counts:\n"+str(df['Target'].value_counts()))
print("\nCount of null values per column:\n"+str(df.isnull().sum()))

Target values:
[0 4]

Target counts:
0    800000
4    800000
Name: Target, dtype: int64

Count of null values per column:
Target    0
ID        0
Date      0
Flag      0
User      0
Text      0
dtype: int64


In [4]:
X = df['Text'].values
Y = df['Target'].values

In [5]:
X

array(["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
       "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!",
       '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds',
       ..., 'Are you ready for your MoJo Makeover? Ask me for details ',
       'Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur ',
       'happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H '],
      dtype=object)

In [6]:
Y

array([0, 0, 0, ..., 4, 4, 4])

#### Training and Testing

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state=32)

#### Vectorization

In [8]:
vect = CountVectorizer()
vect.fit(x_train)

X_train = vect.transform(x_train)
X_test = vect.transform(x_test)
#print(x_train[4])
#print(X_train[4])

In [9]:
print(X_train[0])

  (0, 56129)	1
  (0, 57447)	1
  (0, 126962)	1
  (0, 129411)	1
  (0, 132556)	1
  (0, 142883)	1
  (0, 226884)	1
  (0, 252021)	1
  (0, 256925)	1
  (0, 257741)	1
  (0, 257883)	1
  (0, 433510)	1
  (0, 455368)	1
  (0, 467486)	1
  (0, 486108)	1
  (0, 501574)	1
  (0, 517177)	1
  (0, 520059)	1
  (0, 528806)	1
  (0, 558368)	1
  (0, 562011)	1
  (0, 562867)	1
  (0, 566030)	1
  (0, 566811)	1


In [10]:
symbols = ",.!?/&-:;@'..."
"["+"\\".join(symbols)+"]"

s = x_train[0]
s = ' '.join(x for x in re.split("["+"\\".join(symbols)+"]", s) if x)

for x in s.split():
    if len(x) > 1:
        print(f"{x : <10}{vect.vocabulary_[x.lower()] : ^10}")

It          257883  
was         558368  
rainy       433510  
and         57447   
cloudy      129411  
in          252021  
the         517177  
Windy       566030  
City        126962  
today       528806  
amp         56129   
WF          562011  
customers   142883  
had         226884  
some        486108  
serious     467486  
SAD         455368  
issues      257741  
with        566811  
them        520059  
when        562867  
is          256925  
summer      501574  
coming      132556  


#### Modeling

In [11]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

/Users/marknatavio/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [12]:
score = classifier.score(X_test, y_test)
print(score)

0.80031875


In [13]:
unique = df['Target'].unique()
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

,0,4
0,126496,33825
4,30073,129606


#### Count Vectorization and Modeling

In [14]:
NB = make_pipeline(CountVectorizer(), LogisticRegression())
NB.fit(x_train, y_train)
score = NB.score(x_test, y_test)
print(score)

/Users/marknatavio/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7983125


In [15]:
unique = df['Target'].unique()
y_pred = NB.predict(x_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

,0,4
0,125626,34695
4,29845,129834


#### Tfidf Vectorization and Modeling

In [16]:
NB = make_pipeline(TfidfVectorizer(), LogisticRegression())
NB.fit(x_train, y_train)
score = NB.score(x_test, y_test)
print(score)

unique = df['Target'].unique()
y_pred = NB.predict(x_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

/Users/marknatavio/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.802515625


,0,4
0,127017,33304
4,29891,129788


<hr/>

# Naive Bayes

Here I will be using Naive bayes for text classification

#### Importing

In [17]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix

#### Data

In [18]:
df = pd.read_csv('/Users/marknatavio/Desktop/Final project/Codes/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-15",names=['Target','ID','Date','Flag','User','Text'])
df.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


#### Insights

In [19]:
targets = df['Target'].unique()
print("Target values:\n"+str(targets))
print("\nTarget counts:\n"+str(df['Target'].value_counts()))
print("\nCount of null values per column:\n"+str(df.isnull().sum()))

Target values:
[0 4]

Target counts:
0    800000
4    800000
Name: Target, dtype: int64

Count of null values per column:
Target    0
ID        0
Date      0
Flag      0
User      0
Text      0
dtype: int64


In [20]:
X = df['Text'].values
Y = df['Target'].values

In [21]:
X

array(["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
       "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!",
       '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds',
       ..., 'Are you ready for your MoJo Makeover? Ask me for details ',
       'Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur ',
       'happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H '],
      dtype=object)

In [22]:
Y

array([0, 0, 0, ..., 4, 4, 4])

#### Training and testing

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state=32)

In [24]:
print(x_train[5])

@kgasso Jeez...  Not that it would have made it easier either way...


#### Count Vectorization & Modeling

In [25]:
NB = make_pipeline(CountVectorizer(), MultinomialNB())
NB.fit(x_train, y_train)
score = NB.score(x_test, y_test)
print(score)

0.780703125


In [26]:
unique = df['Target'].unique()
y_pred = NB.predict(x_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

,0,4
0,131110,29211
4,40964,118715


In [27]:
vect = CountVectorizer()
vect.fit(x_train)

X_train = vect.transform(x_train)
X_test = vect.transform(x_test)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print(score)

unique = df['Target'].unique()
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

0.780703125


,0,4
0,131110,29211
4,40964,118715


#### Tfidf Vectorization & Modeling

In [28]:
NB = make_pipeline(TfidfVectorizer(), MultinomialNB())
NB.fit(x_train, y_train)
score = NB.score(x_test, y_test)
print(score)

0.773678125


In [29]:
unique = df['Target'].unique()
y_pred = NB.predict(x_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

,0,4
0,131229,29092
4,43331,116348


<hr/>

# TensorFlow Neural Network

Here I will be using Naive bayes for text classification

In [30]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [31]:
df = pd.read_csv('/Users/marknatavio/Desktop/Final project/Codes/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-15",names=['Target','ID','Date','Flag','User','Text'])
df

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [32]:
data = df[["Text","Target"]]

In [33]:
data["Target"][data["Target"]==4]=1

In [34]:
data_pos = data[data["Target"]==1].iloc[:int(20000)]
data_neg = data[data["Target"]==0].iloc[:int(20000)]

In [35]:
data = pd.concat([data_pos, data_neg])

In [36]:
data["Text"]=data["Text"].str.lower()

In [37]:
stop = stopwords.words('english')

In [38]:
from nltk.corpus import stopwords

stop_words = set(stop)

def clean_words(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

data["Text"] = data["Text"].apply(lambda text: clean_words(text))
data["Text"].head()

800000                love @health4uandpets u guys r best!!
800001    im meeting one besties tonight! cant wait!! - ...
800002    @darealsunisakim thanks twitter add, sunisa! g...
800003    sick really cheap hurts much eat real food plu...
800004                      @lovesbrooklyn2 effect everyone
Name: Text, dtype: object

In [39]:
punctuations = string.punctuation

def clean_punc(text):
    return text.translate(str.maketrans('','', punctuations))

data["Text"] = data["Text"].apply(lambda text: clean_punc(text))
data["Text"].head()

800000                   love health4uandpets u guys r best
800001    im meeting one besties tonight cant wait  girl...
800002    darealsunisakim thanks twitter add sunisa got ...
800003    sick really cheap hurts much eat real food plu...
800004                       lovesbrooklyn2 effect everyone
Name: Text, dtype: object

In [40]:
def clean_repeats(text):
    return re.sub(r'(.)\1+', r'\1', text)

data["Text"] = data["Text"].apply(lambda text: clean_repeats(text))
data["Text"].head()

800000                   love health4uandpets u guys r best
800001    im meting one besties tonight cant wait girl talk
800002    darealsunisakim thanks twiter ad sunisa got me...
800003    sick realy cheap hurts much eat real fod plus ...
800004                         lovesbroklyn2 efect everyone
Name: Text, dtype: object

In [41]:
def clean_email(text):
    return re.sub('@[^\s]+',' ', text)

data["Text"] = data["Text"].apply(lambda text: clean_email(text))
data["Text"].head()

800000                   love health4uandpets u guys r best
800001    im meting one besties tonight cant wait girl talk
800002    darealsunisakim thanks twiter ad sunisa got me...
800003    sick realy cheap hurts much eat real fod plus ...
800004                         lovesbroklyn2 efect everyone
Name: Text, dtype: object

In [42]:
def clean_URLs(text):
    return re.sub('((www\.[^\s]+) | (https?://[^\s]+))','',text)

data["Text"] = data["Text"].apply(lambda text: clean_URLs(text))
data["Text"].head()

800000                   love health4uandpets u guys r best
800001    im meting one besties tonight cant wait girl talk
800002    darealsunisakim thanks twiter ad sunisa got me...
800003    sick realy cheap hurts much eat real fod plus ...
800004                         lovesbroklyn2 efect everyone
Name: Text, dtype: object

In [43]:
def clean_nums(text):
    return re.sub('[0-9]+','',text)

data["Text"] = data["Text"].apply(lambda text: clean_nums(text))
data["Text"].head()

800000                    love healthuandpets u guys r best
800001    im meting one besties tonight cant wait girl talk
800002    darealsunisakim thanks twiter ad sunisa got me...
800003    sick realy cheap hurts much eat real fod plus ...
800004                          lovesbroklyn efect everyone
Name: Text, dtype: object

#### Tokenization

In [44]:
token = RegexpTokenizer(r'\w+')
data["Text"] = data["Text"].apply(token.tokenize)
data["Text"].head()

800000             [love, healthuandpets, u, guys, r, best]
800001    [im, meting, one, besties, tonight, cant, wait...
800002    [darealsunisakim, thanks, twiter, ad, sunisa, ...
800003    [sick, realy, cheap, hurts, much, eat, real, f...
800004                      [lovesbroklyn, efect, everyone]
Name: Text, dtype: object

#### Stemming

In [45]:
st = nltk.PorterStemmer()

def stemming(text):
    return [st.stem(word) for word in text]

data["Text"] = data["Text"].apply(lambda text: stemming(text))
data["Text"].head()

800000               [love, healthuandpet, u, guy, r, best]
800001    [im, mete, one, besti, tonight, cant, wait, gi...
800002    [darealsunisakim, thank, twiter, ad, sunisa, g...
800003    [sick, reali, cheap, hurt, much, eat, real, fo...
800004                       [lovesbroklyn, efect, everyon]
Name: Text, dtype: object

#### Lemmatization

In [46]:
import nltk
nltk.download('wordnet')

lm = nltk.WordNetLemmatizer()

def lemmatizing(text):
    return [lm.lemmatize(word) for word in text]

data["Text"] = data["Text"].apply(lambda text: lemmatizing(text))
data["Text"].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marknatavio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


800000               [love, healthuandpet, u, guy, r, best]
800001    [im, mete, one, besti, tonight, cant, wait, gi...
800002    [darealsunisakim, thank, twiter, ad, sunisa, g...
800003    [sick, reali, cheap, hurt, much, eat, real, fo...
800004                       [lovesbroklyn, efect, everyon]
Name: Text, dtype: object

#### Modeling

In [47]:
X = data["Text"]
y = data["Target"]

In [48]:
max_len = 500
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(sequences_matrix, y, test_size=0.20, random_state=2)

In [50]:
def tensorflow_based_model(): #Defined tensorflow_based_model function for training tenforflow based model
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(2000,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer) 
    model = Model(inputs=inputs,outputs=layer)
    return model

In [51]:
model = tensorflow_based_model() # here we are calling the function of created model

2022-05-08 00:15:14.004288: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [52]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [53]:
history=model.fit(X_train,Y_train,batch_size=80,epochs=6, validation_split=0.1)

Epoch 1/6
360/360 [==============================] - 123s 334ms/step - loss: 0.5686 - accuracy: 0.7002 - val_loss: 0.5264 - val_accuracy: 0.7394
Epoch 2/6
360/360 [==============================] - 121s 336ms/step - loss: 0.5043 - accuracy: 0.7557 - val_loss: 0.5261 - val_accuracy: 0.7387
Epoch 3/6
360/360 [==============================] - 120s 333ms/step - loss: 0.5010 - accuracy: 0.7594 - val_loss: 0.5251 - val_accuracy: 0.7397
Epoch 4/6
360/360 [==============================] - 132s 368ms/step - loss: 0.4731 - accuracy: 0.7753 - val_loss: 0.5319 - val_accuracy: 0.7378
Epoch 5/6
360/360 [==============================] - 131s 364ms/step - loss: 0.4606 - accuracy: 0.7806 - val_loss: 0.5446 - val_accuracy: 0.7347
Epoch 6/6
360/360 [==============================] - 129s 357ms/step - loss: 0.4498 - accuracy: 0.7881 - val_loss: 0.5565 - val_accuracy: 0.7303


In [54]:
accr1 = model.evaluate(X_test,Y_test)

250/250 [==============================] - 11s 43ms/step - loss: 0.5339 - accuracy: 0.7440


In [55]:
print(accr1[1])

0.7440000176429749
